In [ ]:
# ==============================================================================
# ## 1. Setup and Initial Imports
# ==============================================================================
import pandas as pd
from tabulate import tabulate
import os 

# Define constants for file paths
OZONE_PATH_2001_2005 = "/mnt/c/Users/WSTATION/Desktop/GEOAI_ML/data/Daily_Census_Tract-Level_Ozone_Concentrations_2001-2005.csv"
OZONE_PATH_2006_2010 = "/mnt/c/Users/WSTATION/Desktop/GEOAI_ML/data/Daily_Census_Tract-Level_Ozone_Concentrations_2006-2010.csv"
OZONE_PATH_2011_2014 = "/mnt/c/Users/WSTATION/Desktop/GEOAI_ML/data/Daily_Census_Tract-Level_Ozone_Concentrations_2011-2014.csv"
SHP_PATH = "/mnt/c/Users/WSTATION/Desktop/GEOAI_ML/data/counties_shapefile/tl_2010_us_county10.shp"
OUTPUT_DIR = "/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data" # For saving processed files


os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Setup complete. File paths are defined.")

# NOTE: Will provide the original CSV files seperately through download link given how big they are 

In [ ]:
# ==============================================================================
# ## 1. Load Raw Ozone Data
# Description: Loading the three separate ozone datasets
# ==============================================================================
df_ozone_01_05 = pd.read_csv(OZONE_PATH_2001_2005)
df_ozone_06_10 = pd.read_csv(OZONE_PATH_2006_2010)
df_ozone_11_14 = pd.read_csv(OZONE_PATH_2011_2014)

print("Raw ozone datasets loaded.")

# Display head for each loaded dataframe
print("\n---------- df_ozone_01_05 (2001-2005) Head ----------")
print(tabulate(df_ozone_01_05.head(), headers='keys', tablefmt='pretty', showindex=False))

print("\n---------- df_ozone_06_10 (2006-2010) Head ----------")
print(tabulate(df_ozone_06_10.head(), headers='keys', tablefmt='pretty', showindex=False))

print("\n---------- df_ozone_11_14 (2011-2014) Head ----------")
print(tabulate(df_ozone_11_14.head(), headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
# ==============================================================================
# ## 2. Initial Inspection and Standardization of Ozone DataFrames
# Description: Standardizing column names, date formats, and creating a consistent
#              5-digit county identifier (GEOID10). Inspecting key ozone columns.
# ==============================================================================

# Store dataframes in a dictionary for easier batch processing
ozone_dfs = {
    "01_05": df_ozone_01_05,
    "06_10": df_ozone_06_10,
    "11_14": df_ozone_11_14
}

processed_ozone_dfs = {}

for key, df_original in ozone_dfs.items():
    df = df_original.copy() # Work on a copy
    print(f"\n--- Processing DataFrame for period: {key} ---")

    # Standardize GEOID10 creation
    if key == "06_10":   
        # For 2006-2010 data: statefips (2-digit) + countyfips (3-digit part)
        df['statefips_str'] = df['statefips'].astype(str).str.zfill(2)
        df['countyfips_part_str'] = df['countyfips'].astype(str).str.zfill(3)
        df['GEOID10'] = df['statefips_str'] + df['countyfips_part_str']
        # Drop temporary columns
        df.drop(columns=['statefips_str', 'countyfips_part_str'], inplace=True)
        print(f"Created GEOID10 for {key} by combining state and county FIPS parts.")
    else:
        # For 2001-2005 and 2011-2014 data: countyfips is the 5-digit FIPS
        df['GEOID10'] = df['countyfips'].astype(str).str.zfill(5)
        print(f"Standardized GEOID10 for {key} from existing 5-digit countyfips.")

    # Rename ozone prediction/std columns if necessary (specifically for 2011-2014 df)
    if 'ds_o3_pred' in df.columns:  
        df.rename(columns={'ds_o3_pred': 'DS_O3_pred', 'ds_o3_stdd': 'DS_O3_stdd'}, inplace=True)
        print(f"Standardized ozone column names for {key}.")

    # Convert 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])
    print(f"Converted 'date' column to datetime for {key}. Dtype: {df['date'].dtype}")

    # Inspect DS_O3_pred column
    print(f"\nInspecting DS_O3_pred for {key}:")
    if 'DS_O3_pred' in df.columns:
        print(f"  Missing values: {df['DS_O3_pred'].isnull().sum()}")
        print(f"  Descriptive statistics:\n{df['DS_O3_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])}")
    else:
        print(f"  DS_O3_pred column not found in {key} after potential rename.")

    # Check unique GEOID10 counts per year
    print(f"\nUnique GEOID10 counts per year for {key}:")
    print(df.groupby('year')['GEOID10'].nunique())
    
    processed_ozone_dfs[key] = df

# Assign back to original variable names for clarity if needed later or keep in dict
df_ozone_01_05_processed = processed_ozone_dfs["01_05"]
df_ozone_06_10_processed = processed_ozone_dfs["06_10"]
df_ozone_11_14_processed = processed_ozone_dfs["11_14"]

print("\n--- Standardized DataFrames Preview ---")
print("\n---------- df_ozone_01_05_processed Head ----------")
print(tabulate(df_ozone_01_05_processed.head(), headers='keys', tablefmt='pretty', showindex=False))
print("\n---------- df_ozone_06_10_processed Head ----------")
print(tabulate(df_ozone_06_10_processed.head(), headers='keys', tablefmt='pretty', showindex=False))
print("\n---------- df_ozone_11_14_processed Head ----------")
print(tabulate(df_ozone_11_14_processed.head(), headers='keys', tablefmt='pretty', showindex=False))

In [ ]:
# ==============================================================================
# ## 3. Final Column Cleaning of Ozone DataFrames
# Description: Dropping original FIPS columns now that GEOID10 is the standard
# ==============================================================================

# Update the dictionary keys to reflect processed dataframes
ozone_dfs_processed_for_final_cleaning = {
    "01_05": df_ozone_01_05_processed,
    "06_10": df_ozone_06_10_processed,
    "11_14": df_ozone_11_14_processed
}

for key, df in ozone_dfs_processed_for_final_cleaning.items():

    cols_to_drop = ['countyfips', 'statefips']
    existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    if existing_cols_to_drop:
        df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
        print(f"Dropped original FIPS columns {existing_cols_to_drop} from DataFrame for period: {key}")

    print(f"\n--- {key} DataFrame after final column cleaning ---")
    print(f"Columns: {df.columns.tolist()}")
    print(tabulate(df.head(), headers='keys', tablefmt='pretty', showindex=False))
    
    mem_usage_gb = df.memory_usage(deep=True).sum() / (1024 ** 3)
    print(f"Memory usage for {key} (processed): {mem_usage_gb:.3f} GB")


df_ozone_01_05_final = ozone_dfs_processed_for_final_cleaning["01_05"]
df_ozone_06_10_final = ozone_dfs_processed_for_final_cleaning["06_10"]
df_ozone_11_14_final = ozone_dfs_processed_for_final_cleaning["11_14"]

In [ ]:
# ==============================================================================
# ## 4. Save Processed Ozone DataFrames
# Description: Saving the cleaned and standardized ozone data back to feather files.
# ==============================================================================
df_ozone_01_05_final.to_feather(os.path.join(OUTPUT_DIR, "ozone_2001_2005_processed.feather"))
df_ozone_06_10_final.to_feather(os.path.join(OUTPUT_DIR, "ozone_2006_2010_processed.feather"))
df_ozone_11_14_final.to_feather(os.path.join(OUTPUT_DIR, "ozone_2011_2014_processed.feather"))

print("\nProcessed ozone DataFrames saved to feather files in:", OUTPUT_DIR)

##### IMPORTANT ######
# AFTER RUNNING THIS CELL CLEAR ALL OUTPUTS AND RESTART. 
# BY THIS POINT, YOU WOULD HAVE ACCUMULATED 130GB IN SYSTEM RAM. YOU MUST CLEAR MEMORY
# AFTER CLEARING MEMORY, RUN CELL 1 AGAIN, THEN PROCEED TO THE CELL BELOW 

In [ ]:
# ================================================================================
# ## 5. Verification: Re-load and Inspect Saved Files
# Description: Re-loading the processed files to ensure they were saved correctly
# ================================================================================
df_reloaded_01_05 = pd.read_feather(os.path.join(OUTPUT_DIR, "ozone_2001_2005_processed.feather"))
df_reloaded_06_10 = pd.read_feather(os.path.join(OUTPUT_DIR, "ozone_2006_2010_processed.feather"))
df_reloaded_11_14 = pd.read_feather(os.path.join(OUTPUT_DIR, "ozone_2011_2014_processed.feather"))

print("\n--- Verification: Re-loaded DataFrames ---")
print("\nRe-loaded df_ozone_01_05 Head & Dtypes:")
print(tabulate(df_reloaded_01_05.head(), headers='keys', tablefmt='pretty', showindex=False))
print(df_reloaded_01_05.dtypes)

print("\nRe-loaded df_ozone_06_10 Head & Dtypes:")
print(tabulate(df_reloaded_06_10.head(), headers='keys', tablefmt='pretty', showindex=False))
print(df_reloaded_06_10.dtypes)

print("\nRe-loaded df_ozone_11_14 Head & Dtypes:")
print(tabulate(df_reloaded_11_14.head(), headers='keys', tablefmt='pretty', showindex=False))
print(df_reloaded_11_14.dtypes)

print("\nVerification step complete.")

In [ ]:
# ==============================================================================
# ## 6. Detailed Inspection of DS_O3_pred and GEOID10 Coverage
# Description: Performing statistical summaries for the primary ozone
#              measurement column (DS_O3_pred) and verifying the spatial coverage
#              (unique GEOID10 counts per year) for each ozone data period
# ==============================================================================
import pandas as pd 
from tabulate import tabulate # For pretty printing dataframes



ozone_dataframes_for_inspection = {
    "2001-2005": df_reloaded_01_05, 
    "2006-2010": df_reloaded_06_10, 
    "2011-2014": df_reloaded_11_14
}

for period, df in ozone_dataframes_for_inspection.items():
    print(f"\n--- Inspection for Ozone Data: Period {period} ---")
    
    # Inspect DS_O3_pred column
    if 'DS_O3_pred' in df.columns:
        print(f"Missing values in DS_O3_pred: {df['DS_O3_pred'].isnull().sum()}")
        print("Descriptive statistics for DS_O3_pred:")
        print(df['DS_O3_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))
    else:
        print(f"DS_O3_pred column not found in DataFrame for period {period}.")
        
    # Verify unique GEOID10 counts per year
    if 'GEOID10' in df.columns and 'year' in df.columns:
        print("\nUnique GEOID10 counts per year:")
        print(df.groupby('year')['GEOID10'].nunique())
    else:
        print(f"GEOID10 or year column not found for period {period} for GEOID10 count check.")

In [ ]:
# ==============================================================================
# ## 7. Load and Subset Counties Shapefile 
# Description: Loading county boundary data for potential geospatial analysis
#              This step is separate from the main ozone data processing flow
# ==============================================================================
import geopandas as gpd # Placed here as it's specific to this section

try:
    counties_gdf = gpd.read_file(SHP_PATH)

    print("\n--- Counties Shapefile Subset Head ---")
    print(tabulate(counties_gdf.head(), headers='keys', tablefmt='pretty', showindex=False))
except Exception as e:
    print(f"Could not load or process shapefile: {e}")

# Keep only relevant columns (GEOID10, geometry, STATEFP10 renamed to statefips)
counties_gdf_subset = counties_gdf[['GEOID10', 'STATEFP10', 'NAME10', 'geometry']].rename(columns={'STATEFP10': 'statefips'})

# Merge ozone data (df_reloaded_01_05) with the shapefile data on GEOID
merged_gdf = pd.merge(df_reloaded_01_05, counties_gdf_subset, left_on='GEOID10', right_on='GEOID10', how='left')

# Convert the merged DataFrame back to a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry', crs=counties_gdf.crs)

# Display to confirm the merge
print(merged_gdf.head())

In [ ]:
# ==============================================================================
# ## 8. Investigation of Extreme Ozone Event (2001-2005 Data)
# Description: Focusing on the 2001-2005 dataset to identify, detail, and
#              visualize an anomalous high ozone event, particularly in Missouri
#              on September 13, 2001
# ==============================================================================
import matplotlib.pyplot as plt
import geopandas as gpd 
import numpy as np 
import matplotlib.patheffects as path_effects # For text effects on map
import matplotlib.animation as animation # For animation
from IPython.display import HTML # For displaying animation in Jupyter

# --- 2.1 Identify and Detail Extreme Ozone Values ---
# Investigating extreme DS_O3_pred values in the 2001-2005 ozone dataset.
print("\n--- Investigating extreme DS_O3_pred values in df_ozone_2001_2005 ---")
# Define a threshold for extreme values
# The maximum observed in this dataset was around 491 ppb
df_extreme_o3_early = merged_gdf[merged_gdf['DS_O3_pred'] > 140]
print(f"Number of records with DS_O3_pred > 140 ppb in 2001-2005 data: {len(df_extreme_o3_early)}")

if not df_extreme_o3_early.empty:
    print("Details of extreme ozone values (2001-2005, sorted by DS_O3_pred):")
    # Displaying key columns for the identified extreme records.
    cols_to_show = ['date', 'GEOID10', 'statefips', 'latitude', 'longitude', 'DS_O3_pred', 'DS_O3_stdd']
    actual_cols_to_show = [col for col in cols_to_show if col in df_extreme_o3_early.columns]
    print(tabulate(df_extreme_o3_early[actual_cols_to_show].sort_values(by='DS_O3_pred', ascending=False).head(20), # Show top 20
                   headers='keys', tablefmt='pretty', showindex=False))
    
    print(f"\nUnique GEOID10s with extreme ozone (>140 ppb): {df_extreme_o3_early['GEOID10'].nunique()}")
    # Further filtering for the specific known anomaly
    df_anomaly_event = df_extreme_o3_early[
        (df_extreme_o3_early['date'] == pd.Timestamp('2001-09-13')) &
        (df_extreme_o3_early['statefips'] == '29') # Missouri
    ]
    print(f"\nNumber of extreme records (>140ppb) in Missouri on 2001-09-13: {len(df_anomaly_event)}")
    if not df_anomaly_event.empty:
         print("Details of Missouri anomaly event (2001-09-13):")
         print(tabulate(df_anomaly_event[actual_cols_to_show].sort_values(by='DS_O3_pred', ascending=False),
                        headers='keys', tablefmt='pretty', showindex=False))

In [ ]:
# --- 8.2.2 Spatial Visualization of Extreme Ozone Event in Missouri ---
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tabulate import tabulate 
import matplotlib.patheffects as path_effects



# 1. Filter the county shapefile for Missouri (STATEFP10 == '29')
# -------------------------------------------------------------
missouri_fp = '29'
gdf_missouri_counties = counties_gdf[counties_gdf['STATEFP10'] == missouri_fp].copy()

if gdf_missouri_counties.empty:
    print(f"No counties found for STATEFP10 = {missouri_fp}")
else:
    print(f"\nFiltered for Missouri counties. Found {len(gdf_missouri_counties)} counties.")

    if 'df_anomaly_event' in locals() and hasattr(df_anomaly_event, 'crs') and df_anomaly_event.crs:
        if gdf_missouri_counties.crs != df_anomaly_event.crs:
            print(f"Transforming Missouri counties CRS from {gdf_missouri_counties.crs} to {df_anomaly_event.crs}...")
            gdf_missouri_counties = gdf_missouri_counties.to_crs(df_anomaly_event.crs)
    else:
        print("Warning: gdf_extreme_o3_early is not defined or has no CRS")


    # 3. Plot Missouri Counties and Overlay Extreme Ozone Points
    # --------------------------------------------------------
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Plot Missouri county boundaries
    gdf_missouri_counties.plot(ax=ax, facecolor='#eaeaea', edgecolor='black', linewidth=0.4)

    # Plot ozone points filtered for Missouri
    mo_o3 = df_anomaly_event[df_anomaly_event['statefips'] == '29']
    mo_o3.plot(
        ax=ax,
        column='DS_O3_pred',
        cmap='OrRd',
        markersize=mo_o3['DS_O3_pred'] / 8,
        edgecolor='black',
        linewidth=0.4,
        alpha=0.8,
        legend=True,
        legend_kwds={
            'label': "DS_O3_pred (ppb)",
            'orientation': "horizontal",
            'shrink': 0.5,
            'pad': 0.05,
            'aspect': 30,
            'anchor': (0.5, -0.2)
        }
    )

    # Label only counties with significant ozone points to avoid clutter
    counties_to_label = gdf_missouri_counties[
        gdf_missouri_counties.intersects(mo_o3.unary_union)
    ]

    # Annotate counties using centroids
    for idx, row in counties_to_label.iterrows():
        centroid = row['geometry'].centroid
        plt.annotate(
            text=row['NAME10'],
            xy=(centroid.x, centroid.y),
            horizontalalignment='center',
            fontsize=7,
            color='black',
            alpha=0.8,
            path_effects=[plt.matplotlib.patheffects.withStroke(linewidth=1, foreground="white")]
        )

    # Set axes limits
    ax.set_xlim([-95.6, -89.2])
    ax.set_ylim([36.4, 40.4])
    ax.set_aspect('equal')

    # Title and labels
    ax.set_title('Extreme Ozone Values on 2001-09-13 Over Missouri Counties', fontsize=16, pad=15)
    ax.set_xlabel("Longitude", fontsize=13)
    ax.set_ylabel("Latitude", fontsize=13)

    # Grid
    ax.grid(True, linestyle='--', alpha=0.4)

    # Improved layout
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.show()

In [ ]:
# --- 8.2.3 Temporal Visualization of Ozone in Missouri around the Event ---
# filtering data for Missouri around the spike of September 13, 2001
#date range is defined for September and part of October 2001 to see context
date_range_spike_viz = pd.date_range(start='2001-09-01', end='2001-10-17')
mo_o3_for_timeseries = merged_gdf[
    (merged_gdf['statefips'] == '29') & # Missouri
    (merged_gdf['date'].isin(date_range_spike_viz))
]

if not mo_o3_for_timeseries.empty:
    print("\nPlotting daily ozone statistics for Missouri around Sep-Oct 2001")
    # Computing daily statistics for the filtered Missouri data
    daily_stats = mo_o3_for_timeseries.groupby('date')['DS_O3_pred'].agg(['mean', 'median', 'max', 'min', 'std']).reset_index()
    # Plotting the data clearly
    plt.figure(figsize=(12, 6))
    # Mean Ozone
    plt.plot(daily_stats['date'], daily_stats['mean'], '-o', color='blue', label='Mean DS_O3_pred')
    # Median Ozone
    plt.plot(daily_stats['date'], daily_stats['median'], '--s', color='green', label='Median DS_O3_pred')
    # Max Ozone
    plt.plot(daily_stats['date'], daily_stats['max'], '-^', color='red', linewidth=2, label='Max DS_O3_pred')
    # spike on September 13, 2001
    spike_date = pd.Timestamp('2001-09-13')
    plt.axvline(x=spike_date, color='purple', linestyle='--', linewidth=1.5, alpha=0.8, label='Sept 13 spike')
    # Labels, title, legend
    plt.title('Daily Ozone Concentrations in Missouri (Sept-Oct 2001)', fontsize=16)
    plt.xlabel('Date', fontsize=13)
    plt.ylabel('DS_O3_pred (ppb)', fontsize=13)
    plt.grid(alpha=0.3)
    plt.legend()
    plt.xticks(rotation=45)

    # Tidy layout
    plt.tight_layout()
    plt.show()

In [ ]:
# --- 8.2.4 Animated Visualization of Ozone in Missouri ---
# This creates an animation

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.animation as animation
import pandas as pd

# Filter data for Missouri for the specific date range
date_range = pd.date_range(start='2001-09-01', end='2001-10-17')
mo_o3_date_filtered = merged_gdf[
    (merged_gdf['statefips'] == '29') &
    (merged_gdf['date'].isin(date_range))
]

# Convert to GeoDataFrame for plotting
gdf_mo_o3_dates = gpd.GeoDataFrame(
    mo_o3_date_filtered,
    geometry=gpd.points_from_xy(mo_o3_date_filtered.longitude, mo_o3_date_filtered.latitude),
    crs="EPSG:4326"
)

# Set up plot
fig, ax = plt.subplots(figsize=(10, 10))
gdf_missouri_counties.plot(ax=ax, facecolor='#eaeaea', edgecolor='black', linewidth=0.4)

ax.set_xlim([-95.6, -89.2])
ax.set_ylim([36.4, 40.4])
ax.set_aspect('equal')
ax.set_xlabel("Longitude", fontsize=13)
ax.set_ylabel("Latitude", fontsize=13)
ax.grid(True, linestyle='--', alpha=0.4)

# Initialize scatter with dummy data (must be numeric arrays)
scatter = ax.scatter(
    [], [], c=[], cmap='OrRd', 
    vmin=mo_o3_date_filtered['DS_O3_pred'].min(),
    vmax=mo_o3_date_filtered['DS_O3_pred'].max(), 
    edgecolor='black', 
    linewidth=0.5, 
    alpha=0.8, 
    s=[])

# colorbar 
cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', shrink=0.5, pad=0.05, aspect=30)
cbar.set_label("DS_O3_pred (ppb)")

title = ax.set_title('', fontsize=16, pad=15)

# update function
def update(day):
    current_date = date_range[day]
    daily_data = gdf_mo_o3_dates[gdf_mo_o3_dates['date'] == current_date]

    if not daily_data.empty:
        xy = np.column_stack([daily_data.geometry.x, daily_data.geometry.y])
        sizes = daily_data['DS_O3_pred'] / 5
        colors = daily_data['DS_O3_pred']

        scatter.set_offsets(xy)
        scatter.set_sizes(sizes)
        scatter.set_array(colors)
    else:
        scatter.set_offsets(np.empty((0, 2)))
        scatter.set_sizes([])
        scatter.set_array([])

    title.set_text(f'Ozone Values in Missouri on {current_date.strftime("%Y-%m-%d")}')
    return scatter, title

# Create and run animation
anim = animation.FuncAnimation(
    fig, update, frames=len(date_range), interval=800, blit=True, repeat=True
)

plt.tight_layout()

# For Jupyter notebook, render the animation as HTML
from IPython.display import HTML
HTML(anim.to_jshtml())

# anim.save('missouri_ozone_2001_0910_0917.gif', writer='imagemagick', fps=2)

In [ ]:
# ==============================================================================
# ## 7. Load and Subset Counties Shapefile 
# Description: Loading county boundary data for potential geospatial analysis.
# duplicate cell to correct data 
# ==============================================================================

# Keep only relevant columns (GEOID10, geometry, STATEFP10 renamed to statefips)
counties_gdf_subset_2 = counties_gdf[['GEOID10', 'STATEFP10']].rename(columns={'STATEFP10': 'statefips'})

# merge ozone data (df_reloaded_01_05) with the shapefile data on GEOID
df_reloaded_01_05 = pd.merge(df_reloaded_01_05, counties_gdf_subset_2, left_on='GEOID10', right_on='GEOID10', how='left')

# confirm the merge
print(df_reloaded_01_05.head())

In [ ]:
# ==============================================================================
# ## 9. Data Capping for 2001-2005 Ozone Data
# Description: Applying a cap to the anomalously high DS_O3_pred values identified
#              on September 13, 2001, in Missouri, and verifying the effect of this capping
# ==============================================================================

# Keep only relevant columns (GEOID10, geometry, STATEFP10 renamed to statefips)
counties_gdf_subset_2 = counties_gdf[['GEOID10', 'STATEFP10']].rename(columns={'STATEFP10': 'statefips'})

# Merge ozone data (df_reloaded_01_05) with the shapefile data on GEOID
df_reloaded_01_05 = pd.merge(df_reloaded_01_05, counties_gdf_subset_2, left_on='GEOID10', right_on='GEOID10', how='left')

# Display to confirm the merge
print(df_reloaded_01_05.head())

print("\n--- Applying Data Cap to df_ozone_2001_2005 ---")
# Define the capping conditions
# These are specific to the identified anomaly: Missouri (statefips '29'), date 2001-09-13, DS_O3_pred > 150 ppb
mask_to_cap = (
    (df_reloaded_01_05['statefips'] == '29') &
    (df_reloaded_01_05['date'] == pd.Timestamp('2001-09-13')) &
    (df_reloaded_01_05['DS_O3_pred'] > 150)
)

num_values_to_cap = mask_to_cap.sum()
print(f"Number of DS_O3_pred values to be capped at 150 ppb: {num_values_to_cap}")

if num_values_to_cap > 0:
    df_reloaded_01_05.loc[mask_to_cap, 'DS_O3_pred'] = 150
    print("Capping applied.")

    # Verification of the capping effect
    print("\nDescriptive statistics for DS_O3_pred in Missouri on 2001-09-13 after capping:")
    verification_df = df_reloaded_01_05[
        (df_reloaded_01_05['statefips'] == '29') &
        (df_reloaded_01_05['date'] == pd.Timestamp('2001-09-13'))
    ]
    print(verification_df['DS_O3_pred'].describe())
    print(f"Max value after capping for this subset: {verification_df['DS_O3_pred'].max()}")
else:
    print("No values met the capping criteria (or data already capped).")

df_reloaded_01_05.drop(['statefips'], axis=1, inplace=True)

print("\n----------df_reloaded_01_05-----------------")
print(tabulate(df_reloaded_01_05.head(), headers='keys', tablefmt='pretty', showindex=False))

print("Combining the three ozone dataframes")
df_ozone_full = pd.concat(
    [df_reloaded_01_05, df_reloaded_06_10, df_reloaded_11_14],
    ignore_index=True
)
print(f"Shape of combined df_ozone_full: {df_ozone_full.shape}")
print(df_ozone_full.head())
df_ozone_full.to_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data/ozone_2001_2014.feather")


In [ ]:
# ==============================================================================
# ## 10. Data Aggragation for Ozone Data
# Description: Aggragating cencus tract ozone to county level 
#             
# ==============================================================================
# uncomment the line below if you did not run the cell above and want to load the existing data set 
# df_ozone_full= pd.read_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data/ozone_2001_2014.feather")

print("\n---------- Column Data Types (df_ozone_full) ----------")
print(df_ozone_full.dtypes)

# print preview tables
print("\n----------df_ozone_full-----------------")
print(tabulate(df_ozone_full.head(), headers='keys', tablefmt='pretty', showindex=False))

# Inspecting DS_O3_pred column in df_ozone_df_ozone_full
print("--- Inspecting DS_O3_pred column in df_ozone_full ---")
print(f"Missing values in DS_O3_pred: {df_ozone_full['DS_O3_pred'].isnull().sum()}")
print("Descriptive statistics for DS_O3_pred (overall):")
print(df_ozone_full['DS_O3_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))


# Verify unique GEOID10 counts per year for df_ozone_full
print("--- Unique GEOID10 counts per year (df_ozone_full) ---")
print(df_ozone_full.groupby('year')['GEOID10'].nunique())


print("\n--- Aggregating Combined Ozone Data to Annual County Level ---")

# make sure 'year' and 'GEOID10' are suitable for grouping
if not (pd.api.types.is_numeric_dtype(df_ozone_full['year']) and 'GEOID10' in df_ozone_full.columns):
    print("ERROR: 'year' is not numeric or 'GEOID10' is missing/incorrect in df_ozone_full.")
    exit()

df_ozone_annual_county = df_ozone_full.groupby(
    ['year', 'GEOID10']  # Group by year and the 5-digit county FIPS
)['DS_O3_pred'].agg(
    mean_ozone_pred='mean',
    median_ozone_pred='median',
    max_ozone_pred='max',
    p95_ozone_pred=lambda x: x.quantile(0.95), # 95th percentile
    std_ozone_pred='std',
    # count_ozone_days=('DS_O3_pred', 'count') # daily tract observations
).reset_index()

# Rename GEOID10 to 'countyfips' 
df_ozone_annual_county.rename(columns={'GEOID10': 'countyfips'}, inplace=True)

print("\nAggregated Annual County Ozone Data (sample):")
print(tabulate(df_ozone_annual_county.head(), headers='keys', tablefmt='pretty', showindex=False))
print(f"Shape of aggregated ozone data: {df_ozone_annual_county.shape}")
print(f"Number of unique county-year observations: {len(df_ozone_annual_county)}")


print("\n---------- Column Data Types (df_ozone_annual_county) ----------")
print(df_ozone_annual_county.dtypes)


# print preview tables
print("\n----------ddf_ozone_annual_county-----------------")
print(tabulate(df_ozone_annual_county.head(), headers='keys', tablefmt='pretty', showindex=False))

# Inspect DS_O3_pred column in df_ozone_annual_county
print("--- Inspecting mean_ozone_pred column in df_ozone_annual_county ---")
print(f"Missing values in mean_ozone_pred: {df_ozone_annual_county['mean_ozone_pred'].isnull().sum()}")
print("Descriptive statistics for Dmean_ozone_pred (overall):")
print(df_ozone_annual_county['mean_ozone_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))


# Veriy unique GEOID10 counts per year for df_ozone_annual_county
print("--- Unique countyfips counts per year (df_ozone_full) ---")
print(df_ozone_annual_county.groupby('year')['countyfips'].nunique())

In [ ]:
# ==============================================================================
# ## 10.1 Final Processing and export
# Description: fix any errors with countyfips and export
#             
# ==============================================================================

# Filter countyfips with exactly 4 characters
# countyfips_4_chars = df_ozone_annual_county[df_ozone_annual_county['countyfips'].str.len() == 4]['countyfips'].unique()

# Inspect sorted list for easier review
# countyfips_4_chars_sorted = sorted(countyfips_4_chars)

# df_ozone_annual_county.to_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data/ozone_counties_2001_2014.feather")

# uncomment and run the code below is their are fips with 4 chars 
# Ensure countyfips is a zero-padded 5-digit string
# df_ozone_annual_county['countyfips'] = df_ozone_annual_county['countyfips'].astype(str).str.zfill(5)

# # Immediately verify the fix
# print(df_ozone_annual_county.head())
# print(df_ozone_annual_county['countyfips'].str.len().unique())

# df_ozone_annual_county.to_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data/ozone_counties_2001_2014.feather")

In [ ]:
# ==============================================================================
# ## 11. CVD Data Processing
# Description: Process the CVD data 
#             
# ==============================================================================
cvd= pd.read_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI_ML/data/cvd.feather")

print("\n---------- Column Data Types (cvd) ----------")
print(cvd.dtypes)

# print preview tables
print("\n----------cvd-----------------")
print(tabulate(cvd.head(), headers='keys', tablefmt='pretty', showindex=False))

# Verify unique GEOID10 counts per year for df_ozone_annual_county
print("--- Unique countyfips counts per year (cvd) ---")
print(cvd.groupby('Year')['LocationID'].nunique())

print("\nCVD Data Info:")
cvd.info(verbose=True, show_counts=True)
print("\nCVD Data Types:")
print(cvd.dtypes)
print("\nMissing values in CVD Data:")
print(cvd.isnull().sum()) # check Data_Value


# Stratification 1
print(f"\n-------------Stratification1-------------") 
print(f"\nUnique GeographicLevels: {cvd['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory1: {cvd['StratificationCategory1'].unique()}")
print(f"\nUnique Stratification1 (sample): {cvd['Stratification1'].unique()[:10]}")

print("---------------------------")

# Stratification 2 
print(f"\n-------------Stratification2-------------") 
print(f"\nUnique GeographicLevels: {cvd['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory2: {cvd['StratificationCategory2'].unique()}")
print(f"\nUnique Stratification2 (sample): {cvd['Stratification2'].unique()[:10]}")

print("---------------------------")

# Stratification 3
print(f"\n-------------Stratification3-------------") 
print(f"\nUnique GeographicLevels: {cvd['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory3: {cvd['StratificationCategory3'].unique()}")
print(f"\nUnique Stratification3 (sample): {cvd['Stratification3'].unique()[:10]}")


In [ ]:
# ==============================================================================
# ## 11. CVD Data Anomaly Sorting
# Description: Sorting out the anomolies in the CVD data
#             
# ==============================================================================

# Count occurrences of "2010 - 2019" and "1999 - 2010"
year_counts = cvd['Year'].value_counts()

anomaly_counts = year_counts[['2010 - 2019', '1999 - 2010']]
print(anomaly_counts)

# Subset with anomalies
cvd_year_anomalies = cvd[cvd['Year'].isin(['2010 - 2019', '1999 - 2010'])]

# Quick preview
print("Preview of anomalous year data:")
print(cvd_year_anomalies.head())

# Investigate unique values to understand what these records contain
print("Unique values in 'Data_Value_Type' for anomalies:")
print(cvd_year_anomalies['Data_Value_Type'].unique())

print("Unique 'Topic' values for anomalies:")
print(cvd_year_anomalies['Topic'].unique())

# Check if specific Stratifications or Data_Value_Types are exclusive to these anomalies
print("Counts per StratificationCategory1 for anomalies:")
print(cvd_year_anomalies['Stratification1'].value_counts())

# Filte cvd data to exclude anomalous year ranges
cvd_no_ranges = cvd[~cvd['Year'].isin(['2010 - 2019', '1999 - 2010'])].copy()

# Verify exclusion worked properly
print("Unique years after exclusion:")
print(cvd_no_ranges['Year'].unique())

print("Count of remaining records:", len(cvd_no_ranges))


print("\nUnique Data_Value_Type after exclusion:")
print(cvd_no_ranges['Data_Value_Type'].unique())


print("\nUnique Topics after exclusion:")
print(cvd_no_ranges['Topic'].unique())

print("\nUnique Stratifications after exclusion:")
print(cvd_no_ranges[['StratificationCategory1', 'StratificationCategory2', 'StratificationCategory3']].drop_duplicates())

# Verify unique ountyfips counts per year for df_ozone_annual_county
print("--- Unique countyfips counts per year (cvd) ---")
print(cvd_no_ranges.groupby('Year')['LocationID'].nunique())

cvd_no_ranges['countyfips'] = cvd_no_ranges['LocationID'].astype(str).str.zfill(5)
cvd_no_ranges['year'] = cvd_no_ranges['Year'].astype(int)

# Counties in Ozone Data
ozone_counties = set(df_ozone_annual_county['countyfips'].unique())

# Counties in CVD Data
cvd_counties = set(cvd_no_ranges['countyfips'].astype(str).str.zfill(5).unique())

# Counties in CVD but not in Ozone
extra_cvd_counties = cvd_counties - ozone_counties

print("Counties present in CVD data but not in ozone data:")
print(sorted(extra_cvd_counties))
print(f"Number of extra counties: {len(extra_cvd_counties)}")


print("\n---------- Column Data Types (df_ozone_annual_county) ----------")
print(df_ozone_annual_county.dtypes)


# Now print preview tables
print("\n----------ddf_ozone_annual_county-----------------")
print(tabulate(df_ozone_annual_county.head(), headers='keys', tablefmt='pretty', showindex=False))

# Inspecting DS_O3_pred column in df_ozone_annual_county
print("--- Inspecting mean_ozone_pred column in df_ozone_annual_county ---")
print(f"Missing values in mean_ozone_pred: {df_ozone_annual_county['mean_ozone_pred'].isnull().sum()}")
print("Descriptive statistics for Dmean_ozone_pred (overall):")
print(df_ozone_annual_county['mean_ozone_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))


# Verify unique GEOID10 counts per year for df_ozone_annual_county
print("--- Unique countyfips counts per year (df_ozone_full) ---")
print(df_ozone_annual_county.groupby('year')['countyfips'].nunique())


print("\nCVD Data Info:")
cvd_no_ranges.info(verbose=True, show_counts=True)
print("\nCVD Data Types:")
print(cvd_no_ranges.dtypes)
print("\nMissing values in CVD Data:")
print(cvd_no_ranges.isnull().sum()) # check Data_Value


# Stratification 1
print(f"\n-------------Stratification1-------------") 
print(f"\nUnique GeographicLevels: {cvd_no_ranges['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd_no_ranges['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd_no_ranges['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd_no_ranges['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd_no_ranges['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd_no_ranges['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory1: {cvd_no_ranges['StratificationCategory1'].unique()}")
print(f"\nUnique Stratification1 (sample): {cvd_no_ranges['Stratification1'].unique()[:10]}")

print("---------------------------")

# Stratification 2 
print(f"\n-------------Stratification2-------------") 
print(f"\nUnique GeographicLevels: {cvd_no_ranges['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd_no_ranges['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd_no_ranges['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd_no_ranges['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd_no_ranges['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd_no_ranges['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory2: {cvd_no_ranges['StratificationCategory2'].unique()}")
print(f"\nUnique Stratification2 (sample): {cvd_no_ranges['Stratification2'].unique()[:10]}")

print("---------------------------")

# Stratification 3
print(f"\n-------------Stratification3-------------") 
print(f"\nUnique GeographicLevels: {cvd_no_ranges['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {cvd_no_ranges['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {cvd_no_ranges['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {cvd_no_ranges['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {cvd_no_ranges['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {cvd_no_ranges['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory3: {cvd_no_ranges['StratificationCategory3'].unique()}")
print(f"\nUnique Stratification3 (sample): {cvd_no_ranges['Stratification3'].unique()[:10]}")


In [ ]:
# ==============================================================================
# ## 11. Merge and Export
# Description: Merge the Ozone and CVD data and export as feather
#             
# ==============================================================================
# cvd_no_ranges.drop(columns=['Year', 'LocationID'], inplace=True)
# cvd_no_ranges['year'] = cvd_no_ranges['year'].astype('int32')
print(cvd_no_ranges.dtypes[['countyfips', 'year']])
print(cvd_no_ranges.head())


merged_df = pd.merge(
    df_ozone_annual_county,
    cvd_no_ranges,
    on=['countyfips', 'year'],
    how='inner'  
)

print("Merged Data Shape:", merged_df.shape)
print(merged_df.head())

# Verify unique GEOID10 counts per year for merged_df
print("--- Unique countyfips counts per year (merged_df) ---")
print(merged_df.groupby('year')['countyfips'].nunique())


print("\nmerged_df Data Info:")
merged_df.info(verbose=True, show_counts=True)
print("\nmerged_df Data Types:")
print(merged_df.dtypes)
print("\nMissing values in merged_df:")
print(merged_df.isnull().sum()) # check Data_Value

merged_df_clean = merged_df.dropna(subset=['Data_Value']).copy()


print("Remaining rows after filtering:", merged_df_clean.shape[0])
print(merged_df_clean.isnull().sum())

merged_df_clean = merged_df_clean.drop(columns=['Data_Value_Footnote_Symbol', 'Data_Value_Footnote'])
print(merged_df_clean.info())

print("\n---------- Column Data Types (merged_df_clean) ----------")
print(merged_df_clean.dtypes)

merged_df_clean.to_feather("/mnt/c/Users/WSTATION/Desktop/GEOAI/project/homework/homework/data/clean_data/ozone_cvd_master.feather")

# Now print preview tables
print("\n----------dmerged_df_clean-----------------")
print(tabulate(merged_df_clean.head(), headers='keys', tablefmt='pretty', showindex=False))

# Inspecting DS_O3_pred column in merged_df_clean
print("--- Inspecting mean_ozone_pred column in merged_df_clean ---")
print(f"Missing values in mean_ozone_pred: {merged_df_clean['mean_ozone_pred'].isnull().sum()}")
print("Descriptive statistics for Dmean_ozone_pred (overall):")
print(merged_df_clean['mean_ozone_pred'].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))


# Verify unique countyfips counts per year for merged_df_clean
print("--- Unique countyfips counts per year (df_ozone_full) ---")
print(merged_df_clean.groupby('year')['countyfips'].nunique())


print("\nCVD Data Info:")
merged_df_clean.info(verbose=True, show_counts=True)
print("\nCVD Data Types:")
print(merged_df_clean.dtypes)
print("\nMissing values in CVD Data:")
print(merged_df_clean.isnull().sum()) # check Data_Value


# Stratification 1
print(f"\n-------------Stratification1-------------") 
print(f"\nUnique GeographicLevels: {merged_df_clean['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {merged_df_clean['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {merged_df_clean['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {merged_df_clean['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {merged_df_clean['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {merged_df_clean['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory1: {merged_df_clean['StratificationCategory1'].unique()}")
print(f"\nUnique Stratification1 (sample): {merged_df_clean['Stratification1'].unique()[:10]}")

print("---------------------------")

# Stratification 2 
print(f"\n-------------Stratification2-------------") 
print(f"\nUnique GeographicLevels: {merged_df_clean['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {merged_df_clean['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {merged_df_clean['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {merged_df_clean['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {merged_df_clean['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {merged_df_clean['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory2: {merged_df_clean['StratificationCategory2'].unique()}")
print(f"\nUnique Stratification2 (sample): {merged_df_clean['Stratification2'].unique()[:10]}")

print("---------------------------")

# Stratification 3
print(f"\n-------------Stratification3-------------") 
print(f"\nUnique GeographicLevels: {merged_df_clean['GeographicLevel'].unique()}") # Expect 'County'
print(f"\nUnique DataSources: {merged_df_clean['DataSource'].unique()}") # Expect 'NVSS'
print(f"\nUnique Classes: {merged_df_clean['Class'].unique()}") # Filter for 'Cardiovascular Diseases'
print(f"\nUnique Topics (sample): {merged_df_clean['Topic'].unique()[:20]}") # To see heart disease types
print(f"\nUnique Data_Value_Units: {merged_df_clean['Data_Value_Unit'].unique()}")
print(f"\nUnique Data_Value_Types: {merged_df_clean['Data_Value_Type'].unique()}") # CRITICAL
print(f"\nUnique StratificationCategory3: {merged_df_clean['StratificationCategory3'].unique()}")
print(f"\nUnique Stratification3 (sample): {merged_df_clean['Stratification3'].unique()[:10]}")